<a href="https://colab.research.google.com/github/murfish/AI-Enhancement-with-Knowledge-Graphs---Mastering-RAG-Systems/blob/main/Aquaculture_Bio_Economic_Model_Workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. CENTRALIZED INPUT PARAMETERS ---
# All assumptions are consolidated here for easy scenario analysis.

def get_parameters():
    """
    Returns a dictionary containing all baseline assumptions for the model.
    This function centralizes all user-configurable parameters.
    """
    params = {
        # --- Biological & Growout Parameters ---
        'initial_stocking_count': 10000,       # Number of fish at Day 0
        'initial_stocking_weight_g': 100,      # Avg weight (grams) at start of growout
        'final_target_weight_g': 1500,         # Target avg weight (grams) at harvest
        'growout_days': 120,                   # Total duration of the growout cycle
        'stanza_breakpoints_g': ,         # Weight points where growth rate changes (e.g., puberty)
        'sgr_decay_factor': 1.25,              # Assumes SGR in a stanza is X times the SGR of the next stanza
        'cumulative_mortality_rate': 0.10,     # 10% total mortality over the cycle

        # --- Cost & Feed Parameters ---
        'fcr': 1.2,                            # Feed Conversion Ratio (kg feed per kg biomass gain)
        'feed_cost_per_kg': 1.50,              # Cost ($) per kg of feed
        'fingerling_cost_per_fish': 0.50,      # Cost per 100g juvenile fish

        # --- Primary Harvest & Revenue Parameters ---
        'premium_grade_percentage': 0.70,      # 70% of harvest is Premium grade
        'premium_price_per_kg': 15.00,         # Sale price ($) for whole Premium fish
        'b_grade_price_per_kg': 10.00,         # Sale price ($) for whole B-graded fish

        # --- Value-Added By-Product Parameters ---
        # Allocations are percentages of the Premium Grade biomass
        'allocation_to_fillet_pct': 0.20,      # 20% of Premium fish go to filleting
        'fillet_yield_pct': 0.45,              # 45% of whole fish weight remains after filleting
        'fillet_processing_cost_per_kg': 2.50, # Cost per kg of *incoming* fish weight
        'fillet_sale_price_per_kg': 35.00,     # Sale price ($) of finished fillets

        'allocation_to_hotsmoke_pct': 0.15,    # 15% of Premium fish go to hot smoking
        'hotsmoke_yield_pct': 0.35,            # 35% of whole fish weight remains after smoking
        'hotsmoke_processing_cost_per_kg': 5.00,# Cost per kg of *incoming* fish weight
        'hotsmoke_sale_price_per_kg': 45.00,   # Sale price ($) of finished smoked product

        'allocation_to_dryage_pct': 0.10,      # 10% of Premium fish go to dry-aging
        'dryage_yield_pct': 0.75,              # 75% of whole fish weight remains after dry-aging
        'dryage_processing_cost_per_kg': 3.00, # Cost per kg of *incoming* fish weight
        'dryage_sale_price_per_kg': 60.00,     # Sale price ($) of finished dry-aged product
    }

    # B-grade percentage is derived from premium, ensuring total is 100%
    params['b_grade_percentage'] = 1.0 - params['premium_grade_percentage']

    return params

# --- 2. GROWTH MODEL CALCULATION ---

def calculate_growth_stanzas(params):
    """
    Calculates the Specific Growth Rates (SGR) and durations for each growth stanza
    to meet the final weight target over the specified growout period.

    This function solves a system of equations based on the SGR formula and the
    assumption that SGR decreases as fish get larger.
    """
    weight_points = [params['initial_stocking_weight_g']] + \
                    params['stanza_breakpoints_g'] + \
                    [params['final_target_weight_g']]

    num_stanzas = len(weight_points) - 1
    sgr_decay = params['sgr_decay_factor']
    total_days = params['growout_days']

    # Calculate the log-weight differences for each stanza
    log_weight_diffs = [np.log(weight_points[i+1]) - np.log(weight_points[i]) for i in range(num_stanzas)]

    # Set up the equation to solve for the SGR of the final stanza (sgr_n)
    # total_days = sum [ log_diff_i / sgr_i ]
    # sgr_i = sgr_n * (decay_factor ^ (n - i))
    denominator_sum = 0
    for i in range(num_stanzas):
        sgr_multiplier = sgr_decay ** (num_stanzas - 1 - i)
        denominator_sum += log_weight_diffs[i] / sgr_multiplier

    sgr_last_stanza = denominator_sum / total_days

    sgrs = [sgr_last_stanza * (sgr_decay ** (num_stanzas - 1 - i)) for i in range(num_stanzas)]

    # Calculate the duration of each stanza
    durations = [int(round(log_weight_diffs[i] / sgrs[i])) for i in range(num_stanzas)]

    # Adjust last stanza duration to ensure total days sum correctly due to rounding
    durations[-1] = total_days - sum(durations[:-1])

    stanzas =
    day_start = 1
    for i in range(num_stanzas):
        stanzas.append({
            'stanza': i + 1,
            'start_day': day_start,
            'end_day': day_start + durations[i] - 1,
            'duration': durations[i],
            'sgr': sgrs[i],
            'start_weight_g': weight_points[i],
            'end_weight_g': weight_points[i+1]
        })
        day_start += durations[i]

    return stanzas

# --- 3. BIOLOGICAL SIMULATION ---

def run_biological_simulation(params, stanzas):
    """
    Simulates the daily growth, mortality, and feed consumption of the fish cohort.
    Returns a pandas DataFrame with the daily simulation results.
    """
    days = params['growout_days']
    df = pd.DataFrame(index=range(1, days + 1))
    df['day'] = df.index

    # Initialize starting conditions
    df.loc[1, 'population'] = params['initial_stocking_count']
    df.loc[1, 'avg_weight_g'] = params['initial_stocking_weight_g']

    daily_mortality_rate = params['cumulative_mortality_rate'] / days

    # Create a map of day -> sgr for quick lookup
    sgr_map = {}
    for s in stanzas:
        for day in range(s['start_day'], s['end_day'] + 1):
            sgr_map[day] = s['sgr']

    # Run the daily simulation loop
    for day in range(1, days):
        # Apply mortality first
        population_start_of_day = df.loc[day, 'population']
        daily_deaths = population_start_of_day * daily_mortality_rate
        df.loc[day + 1, 'population'] = population_start_of_day - daily_deaths

        # Apply growth
        sgr = sgr_map.get(day, 0)
        weight_start_of_day = df.loc[day, 'avg_weight_g']
        df.loc[day + 1, 'avg_weight_g'] = weight_start_of_day * np.exp(sgr)

    # Fill day 1 population for calculations
    df['population'] = df['population'].fillna(method='bfill')

    # Calculate derived metrics
    df['total_biomass_kg'] = (df['population'] * df['avg_weight_g']) / 1000
    df['daily_biomass_gain_kg'] = df['total_biomass_kg'].diff().fillna(0)
    df.loc[df['daily_biomass_gain_kg'] < 0, 'daily_biomass_gain_kg'] = 0 # No negative gain

    df['daily_feed_kg'] = df['daily_biomass_gain_kg'] * params['fcr']
    df['cumulative_feed_kg'] = df['daily_feed_kg'].cumsum()
    df['daily_feed_cost'] = df['daily_feed_kg'] * params['feed_cost_per_kg']

    return df

# --- 4. FINANCIAL CALCULATION ---

def calculate_financials(params, sim_df):
    """
    Calculates revenues, costs, and profits based on the simulation output.
    Returns a dictionary with the final financial summary.
    """
    # --- Calculate Costs (COGS) ---
    cost_of_fingerlings = params['initial_stocking_count'] * params['fingerling_cost_per_fish']
    total_feed_cost = sim_df['daily_feed_cost'].sum()

    # --- Calculate Harvest and Primary Revenue ---
    final_biomass_kg = sim_df.loc[params['growout_days'], 'total_biomass_kg']

    premium_biomass_kg = final_biomass_kg * params['premium_grade_percentage']
    b_grade_biomass_kg = final_biomass_kg * params['b_grade_percentage']

    # Calculate biomass for whole fish sale (premium not allocated to processing)
    premium_allocation_sum = (params['allocation_to_fillet_pct'] +
                              params['allocation_to_hotsmoke_pct'] +
                              params['allocation_to_dryage_pct'])

    premium_whole_sale_kg = premium_biomass_kg * (1 - premium_allocation_sum)

    revenue_premium_whole = premium_whole_sale_kg * params['premium_price_per_kg']
    revenue_b_grade_whole = b_grade_biomass_kg * params['b_grade_price_per_kg']

    # --- Calculate By-Product Revenues and Costs ---
    results = {}

    # Filleting
    fillet_input_kg = premium_biomass_kg * params['allocation_to_fillet_pct']
    fillet_output_kg = fillet_input_kg * params['fillet_yield_pct']
    results['revenue_fillet'] = fillet_output_kg * params['fillet_sale_price_per_kg']
    results['cost_fillet'] = fillet_input_kg * params['fillet_processing_cost_per_kg']

    # Hot Smoking
    hotsmoke_input_kg = premium_biomass_kg * params['allocation_to_hotsmoke_pct']
    hotsmoke_output_kg = hotsmoke_input_kg * params['hotsmoke_yield_pct']
    results['revenue_hotsmoke'] = hotsmoke_output_kg * params['hotsmoke_sale_price_per_kg']
    results['cost_hotsmoke'] = hotsmoke_input_kg * params['hotsmoke_processing_cost_per_kg']

    # Dry-Aging
    dryage_input_kg = premium_biomass_kg * params['allocation_to_dryage_pct']
    dryage_output_kg = dryage_input_kg * params['dryage_yield_pct']
    results['revenue_dryage'] = dryage_output_kg * params['dryage_sale_price_per_kg']
    results['cost_dryage'] = dryage_input_kg * params['dryage_processing_cost_per_kg']

    # --- Aggregate Financials ---
    total_revenue = (revenue_premium_whole + revenue_b_grade_whole +
                     results['revenue_fillet'] + results['revenue_hotsmoke'] +
                     results['revenue_dryage'])

    total_processing_costs = (results['cost_fillet'] + results['cost_hotsmoke'] +
                              results['cost_dryage'])

    total_cogs = cost_of_fingerlings + total_feed_cost + total_processing_costs

    gross_profit = total_revenue - total_cogs

    # --- Final Summary Dictionary ---
    summary = {
        'Total Revenue': total_revenue,
        'Revenue from Premium Whole Fish': revenue_premium_whole,
        'Revenue from B-Graded Whole Fish': revenue_b_grade_whole,
        'Revenue from Fillets': results['revenue_fillet'],
        'Revenue from Hot Smoked': results['revenue_hotsmoke'],
        'Revenue from Dry-Aged': results['revenue_dryage'],

        'Total COGS': total_cogs,
        'Cost of Fingerlings': cost_of_fingerlings,
        'Cost of Feed': total_feed_cost,
        'Total Processing Costs': total_processing_costs,

        'Gross Profit': gross_profit,
        'Gross Margin (%)': (gross_profit / total_revenue) * 100 if total_revenue else 0,

        # Key Performance Indicators (KPIs)
        'KPIs': {
            'Final Harvest Biomass (kg)': final_biomass_kg,
            'Fish Survival Rate (%)': (sim_df.loc[days, 'population'] / params['initial_stocking_count']) * 100,
            'Final Average Weight (g)': sim_df.loc[days, 'avg_weight_g'],
            'Total Feed Used (kg)': sim_df['cumulative_feed_kg'].max(),
            'Revenue per Fish Stocked ($)': total_revenue / params['initial_stocking_count'],
            'Profit per Fish Stocked ($)': gross_profit / params['initial_stocking_count'],
            'Peak Working Capital Required ($)': (cost_of_fingerlings + total_feed_cost)
        }
    }

    return summary

# --- 5. REPORTING & VISUALIZATION ---

def print_summary_report(summary, stanzas):
    """Prints a formatted summary of the financial results and KPIs."""
    print("\n" + "="*60)
    print("BIO-ECONOMIC MODEL: COHORT FINANCIAL SUMMARY")
    print("="*60)

    print("\n--- Growth Stanza Structure ---")
    for s in stanzas:
        print(f"Stanza {s['stanza']}: Days {s['start_day']}-{s['end_day']} | "
              f"SGR: {s['sgr']*100:.2f}%/day | "
              f"Weight: {s['start_weight_g']:.0f}g to {s['end_weight_g']:.0f}g")

    print("\n--- Revenue Breakdown ---")
    for key, value in summary.items():
        if 'Revenue' in key:
            print(f"{key:<35}: ${value:12,.2f}")

    print("\n--- Cost of Goods Sold (COGS) Breakdown ---")
    for key, value in summary.items():
        if 'Cost' in key:
            print(f"{key:<35}: ${value:12,.2f}")

    print("\n--- Profitability ---")
    print(f"{'Gross Profit':<35}: ${summary['Gross Profit']:12,.2f}")
    print(f"{'Gross Margin (%)':<35}: {summary['Gross Margin (%)']:11,.2f}%")

    print("\n" + "-"*60)
    print("KEY PERFORMANCE INDICATORS (KPIs)")
    print("-"*60)
    for key, value in summary['KPIs'].items():
        if '$' in key:
            print(f"{key:<35}: ${value:12,.2f}")
        else:
            print(f"{key:<35}: {value:12,.2f}")
    print("="*60)

def plot_simulation_results(sim_df):
    """Generates and displays plots for key simulation metrics."""
    sns.set_style("whitegrid")
    fig, axes = plt.subplots(3, 1, figsize=(12, 18))
    fig.suptitle('Cohort Growout Simulation Results', fontsize=16, y=0.92)

    # Plot 1: Average Weight
    sns.lineplot(ax=axes, x=sim_df['day'], y=sim_df['avg_weight_g'], color='blue')
    axes.set_title('Average Individual Weight Over Time')
    axes.set_xlabel('Day of Growout Cycle')
    axes.set_ylabel('Average Weight (grams)')

    # Plot 2: Total Biomass
    sns.lineplot(ax=axes[1], x=sim_df['day'], y=sim_df['total_biomass_kg'], color='green')
    axes.[1]set_title('Total Cohort Biomass Over Time')
    axes.[1]set_xlabel('Day of Growout Cycle')
    axes.[1]set_ylabel('Total Biomass (kg)')

    # Plot 3: Cumulative Costs
    sim_df['cumulative_cost'] = sim_df['daily_feed_cost'].cumsum() + \
                                (get_parameters()['initial_stocking_count'] * get_parameters()['fingerling_cost_per_fish'])
    sns.lineplot(ax=axes[2], x=sim_df['day'], y=sim_df['cumulative_cost'], color='red')
    axes.[2]set_title('Cumulative Production Cost (Fingerlings + Feed)')
    axes.[2]set_xlabel('Day of Growout Cycle')
    axes.[2]set_ylabel('Cumulative Cost ($)')

    plt.tight_layout(rect=[0, 0, 1, 0.9])
    plt.show()

# --- 6. MAIN EXECUTION ---

if __name__ == "__main__":
    # 1. Load parameters
    params = get_parameters()

    # 2. Calculate the growth model based on targets
    stanzas = calculate_growth_stanzas(params)

    # 3. Run the biological simulation
    simulation_results_df = run_biological_simulation(params, stanzas)

    # 4. Calculate the financial outcomes
    financial_summary = calculate_financials(params, simulation_results_df)

    # 5. Print the summary report
    print_summary_report(financial_summary, stanzas)

    # 6. Generate and display plots
    plot_simulation_results(simulation_results_df)